# CouncilEval
- A collaborative evaluation framework where multiple expert agents convene to assess, compare, and refine recommendations. By simulating a panel of specialists—each focusing on different performance criteria—CouncilEval ensures well-rounded, consensus-driven decisions on system recommendations, ultimately delivering more balanced, reliable, and user-focused outcomes.

In [8]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [9]:
from typing import Dict, List, TypedDict
from langgraph.graph import StateGraph, END, START
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
import json
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Optional
import re
import rich
def convert_messages_to_serializable(obj):
    if isinstance(obj, dict):
        # 遞迴處理字典
        return {k: convert_messages_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        # 遞迴處理列表
        return [convert_messages_to_serializable(v) for v in obj]
    elif isinstance(obj, HumanMessage) or isinstance(obj, SystemMessage):
        # 將 Message 物件轉成字典格式
        # ontent 前面有 "Speaker: 內容" 格式，可以分離
        parts = obj.content.split(":", 1)
        if len(parts) == 2:
            speaker = parts[0].strip()
            message = parts[1].strip()
        else:
            speaker = "unknown"
            message = obj.content
        
        return {
            "speaker": speaker,
            "message": message
        }
    else:
        # 對於非字典/列表/Message物件
        return obj

class EvaluationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "聊天歷史"]
    next_agent: str
    question_id: int
    question: str
    gpt4o_recommendation: str
    buildmate_recommendation: str
    agent_scores: Dict[str, Dict[str, int]]
    round_count: int     # 用於紀錄第幾輪
    max_rounds: int       # 最大回合數
    consensus_reached: bool  # 是否已達成共識

def create_agent(name: str, llm):
    # new 
    system_prompts = {
    "Agent1": """你是電腦硬體顧問，專精於「效能」與「系統穩定」層面。
請從處理器、顯示卡、記憶體、儲存裝置的整體效能，以及散熱方案、電源品質、系統組裝平衡度，來比較兩個推薦清單的效能與穩定度。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Agent2": """你是電腦硬體顧問，專精於「價格/效能比」(CP值)。
請根據兩個推薦清單中各零件的價位與提供效能是否相稱、預算是否用在刀口上，評估CP值高低。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Agent3": """你是電腦硬體顧問，專精於「需求符合度」。
1.請根據使用者的需求場景（例如遊戲、剪輯、多工、深度學習、家庭多媒體等），判斷各推薦清單是否選用合適的CPU/GPU、記憶體容量、儲存空間及其他週邊，並評估清單是否過度或不足。
2.請評估使用者的總預算與清單總價是否接近，在使用者預算合理下，應該要充分運用預算。
3.請評估清單是否與使用者的需求相符，例如零組件外觀，主機大小，保固等各式需求。
4.請評估使用者的特殊考量事項是否被滿足，例如：省電。

請提供：
[對gpt4o評價]: (0~10)
[對BuildMate評價]: (0~10)
[簡短原因]: (以專業觀點精簡描述)

注意事項：
1. 原因的說明請限制在50個字以內。
""",

    "Coordinator": """你是最終協調者，已獲得3位專家針對效能與穩定、CP值、需求符合度的評分。
如果尚未達成共識，請只簡短回應「未達共識」。
如果已達共識，請回應「已達成共識」並指出哪一方較佳(不超過50字)。"""
}


    def agent_function(state: EvaluationState):
        messages = list(state["messages"])
        question = state["question"]
        gpt4o_recom = state["gpt4o_recommendation"]
        buildmate_recom = state["buildmate_recommendation"]

        system_msg = SystemMessage(content=system_prompts[name])
        user_prompt = f"""(第{state['round_count']}輪)
問題: {question}

--- gpt4o的組裝清單 ---
{gpt4o_recom}

--- BuildMate的組裝清單 ---
{buildmate_recom}
"""

        conversation = [system_msg]
        conversation.extend(messages)
        conversation.append(HumanMessage(content=user_prompt))

        llm_response = llm.invoke(conversation)
        content_with_speaker = f"{name}: {llm_response.content}"
        rich.print( content_with_speaker)
        if name in ("Agent1","Agent2","Agent3"):
            gpt4o_match = re.search(r"\[對gpt4o評價\]:\s*(\d+)", llm_response.content)
            bm_match = re.search(r"\[對BuildMate評價\]:\s*(\d+)", llm_response.content)
            pattern = r"\[簡短原因\]:\s*(.+)"
            reason_match = re.search(pattern, llm_response.content)
            reason = reason_match.group(1)

            if gpt4o_match and bm_match:
                gpt4o_score = int(gpt4o_match.group(1))
                bm_score = int(bm_match.group(1))
                # 將分數存入
                state["agent_scores"][f"{name}_round{state['round_count']}"] = {
                    "gpt4o_score": gpt4o_score,
                    "buildmate_score": bm_score,
                    "reason": reason
                }
                #print (f'gpt score: {gpt4o_score}, buildmate score: {bm_score}')

        if name == "Coordinator":
            # 假設協調者若尚未達成共識則回應「未達共識」四字，如果達成共識就「已達成共識」
            if "已達成共識" in llm_response.content:
                state["consensus_reached"] = True
            else:
                state["consensus_reached"] = False

        new_message = HumanMessage(content=content_with_speaker)

        return {
            "messages": [*messages, new_message],
            "next_agent": decide_next_speaker(name, state),
            "question_id": state["question_id"],
            "question": state["question"],
            "gpt4o_recommendation": state["gpt4o_recommendation"],
            "buildmate_recommendation": state["buildmate_recommendation"],
            "agent_scores": state["agent_scores"],
            "round_count": state["round_count"],
            "max_rounds": state["max_rounds"],
            "consensus_reached": state["consensus_reached"]
        }
    return agent_function


def decide_next_speaker(current_speaker: str, state: EvaluationState) -> str:
    if current_speaker == "Agent1":
        return "Agent2"
    elif current_speaker == "Agent2":
        return "Agent3"
    elif current_speaker == "Agent3":
        return "Coordinator"
    elif current_speaker == "Coordinator":
        # 即使已達共識，但如果 round_count < 2，仍然強制進入下一輪
        if state["round_count"] < 2:
            # 即使協調者表示「已達成共識」，也不要結束
            state["round_count"] += 1
            state["consensus_reached"] = False  # 忽略本輪的共識結果，強制再跑一輪
            return "Agent1"
        else:
            # 現在已經執行到第二輪或更多
            # 這裡才根據共識及最大回合數決定是否結束
            if state["consensus_reached"] or state["round_count"] >= state["max_rounds"]:
                return END
            else:
                state["round_count"] += 1
                return "Agent1"
    else:
        return END

def create_workflow(llm):
    workflow = StateGraph(EvaluationState)
    workflow.add_node("Agent1", create_agent("Agent1", llm))
    workflow.add_node("Agent2", create_agent("Agent2", llm))
    workflow.add_node("Agent3", create_agent("Agent3", llm))
    workflow.add_node("Coordinator", create_agent("Coordinator", llm))

    def router(state: EvaluationState) -> str:
        return state["next_agent"]

    workflow.add_conditional_edges("Agent1", router, {"Agent2": "Agent2", END: END})
    workflow.add_conditional_edges("Agent2", router, {"Agent3": "Agent3", END: END})
    workflow.add_conditional_edges("Agent3", router, {"Coordinator": "Coordinator", END: END})
    workflow.add_conditional_edges("Coordinator", router, {"Agent1": "Agent1", END: END})

    workflow.add_edge(START, "Agent1")

    return workflow.compile()


# 要比較的推薦清單
input_recommendation_file = './final_output_all.json'
with open(input_recommendation_file, "r", encoding="utf-8") as f:
        recommendation_data  = json.load(f)
        final_report = []  # 用來儲存所有 question 的評估結果

        for sample_data in recommendation_data:

            rich.print(sample_data)
            #llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0.3)
            llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)
            app = create_workflow(llm)

            initial_state = {
                "messages": [],
                "next_agent": "Agent1",
                "question_id": sample_data["question_id"],
                "question": sample_data["question"],
                "gpt4o_recommendation": sample_data["gpt4o_recommendation"],
                "buildmate_recommendation": sample_data["buildmate_recommendation"],
                "agent_scores": {},
                "round_count": 1,
                "max_rounds": 3,  #最多3回合
                "consensus_reached": False
            }

            final_state = None
            i = 0 
            for output in app.stream(initial_state):
                print (f"======")
                final_state = output
                i = i + 1
            print (final_state)
            serializable_state = dict(final_state)
            state = final_state["Coordinator"]
            question_id = state["question_id"]
            question = state["question"]
            gpt4o_recom = state["gpt4o_recommendation"]
            buildmate_recom = state["buildmate_recommendation"]
            agent_scores = state["agent_scores"]


            gpt4o_scores_all = []
            buildmate_scores_all = []
            rounds_data = {}

            for key, scores in agent_scores.items():
                m = re.match(r"(Agent\d+)_round(\d+)", key)
                if m:
                    agent_name = m.group(1)
                    round_num = int(m.group(2))
                    if round_num not in rounds_data:
                        rounds_data[round_num] = {}
                    rounds_data[round_num][agent_name] = scores
                    gpt4o_scores_all.append(scores["gpt4o_score"])
                    buildmate_scores_all.append(scores["buildmate_score"])

            if gpt4o_scores_all:
                gpt4o_avg = sum(gpt4o_scores_all) / len(gpt4o_scores_all)
            else:
                gpt4o_avg = 0

            if buildmate_scores_all:
                buildmate_avg = sum(buildmate_scores_all) / len(buildmate_scores_all)
            else:
                buildmate_avg = 0

            # 判斷誰贏 (1: BuildMate 贏; 0: BuildMate 輸)
            winner = 1 if buildmate_avg > gpt4o_avg else 0

            # 整理 round 資訊
            rounds_list = []
            for r in sorted(rounds_data.keys()):
                round_info = {"round_number": r}
                for agent_name, sc in rounds_data[r].items():
                    round_info[agent_name] = {
                        "gpt4o_score": sc["gpt4o_score"],
                        "buildmate_score": sc["buildmate_score"],
                        "reason": sc["reason"]
                    }
                rounds_list.append(round_info)

            report_item = {
                "question_id": question_id,
                "question": question,
                "buildmate_recommendation":buildmate_recom,
                "gpt4o_recommendation": gpt4o_recom,
                "winner": winner,
                "buildmate_avg_score": buildmate_avg,
                "gpt4o_avg_score": gpt4o_avg,
                "rounds": rounds_list
            }

            print(json.dumps(report_item, ensure_ascii=False, indent=2))
            final_report.append(report_item)

            #serializable_result = convert_messages_to_serializable(final_state)
# 輸出評估結果的檔案 
output_report = "evaluation_report_budget_reason_gemini_all.json"
with open(output_report, "w", encoding="utf-8") as out_f:
    json.dump(final_report, out_f, ensure_ascii=False, indent=2)





{
    'question_id': 1,
    'question': '請問30000元內能組一台什麼樣的主機，主要是玩LOL和看Netflix用？',
    'gpt4o_recommendation': 'CPU: AMD Ryzen 5 5600G | price: 4990\nMemory: Corsair Vengeance LPX 16GB (2 x 8GB) 
DDR4-3200 | price: 1590\nGPU: MSI GeForce GTX 1650 D6 VENTUS XS OC | price: 6490\nStorage: ADATA XPG SX8200 Pro 
512GB NVMe M.2 | price: 1490\nPower Supply: Cooler Master MWE 550W 80+ Bronze | price: 1890\nCase: Cooler Master 
MasterBox Q300L | price: 1290\nCase Fan: ARCTIC P12 PWM PST 120mm | price: 490\n總價: 18230',
    'buildmate_recommendation': 'CPU: Intel i5-14400F | price: 5400\nMotherboard: 華碩 PRIME 
H610M-E-CSM(DDR5)(M-ATX/Realtek 1Gb/註冊四年/2DIMM) | price: 2990\nMemory: 威剛 單條16GB DDR5-5600/CL46 
(AD5U560016G-S)【雙參數】 | price: 1269\nGPU: Acer Nitro Intel ARC B580 OC 12GB | price: 8690\nStorage: 威剛 
Ultimate SU800 512G/2.5吋/讀:560M/寫:520M/TLC/DRAM快取/SMI/台灣製 | price: 1499\nPower Supply: Antec 
HCG1000(1000W)雙8/金牌/全模/全日系/智慧溫控風扇/10年保 送12+4pin模組線！ | price: 3990\nCase: 銀欣 RL08 紅黑 | 
price: 1290\nCase Fan: MSI MEG SILENT GALE P12 靜音風扇 /HDB軸承/2000RPM/抗震襯墊/PWM/六年保 | price: 790\n總價: 
25918'
}

Agent1: [對gpt4o評價]: 7
[對BuildMate評價]: 4
[簡短原因]: gpt4o均衡，價格合理。BuildMate電源過剩，記憶體與主機板不相容。

Agent2: [對gpt4o評價]: 8
[對BuildMate評價]: 5
[簡短原因]: gpt4o配置均衡，價格合理。BuildMate主機板不支援DDR5記憶體，電源瓦數過高。

Agent3: [對gpt4o評價]: 8
[對BuildMate評價]: 4
[簡短原因]: gpt4o符合需求和預算。BuildMate主機板不支援DDR5、電源過剩、整體價格偏高。

Coordinator: 已達成共識，gpt4o較佳。配置均衡價格合理，BuildMate則出現不相容和過剩的配置導致性價比低。

...

Agent1: [對gpt4o評價]: 8
[對BuildMate評價]: 5
[簡短原因]: gpt4o均衡，BuildMate記憶體單通道、B580驅動及相容性堪慮、1000W電源過剩。

Agent2: [對gpt4o評價]: 8
[對BuildMate評價]: 4
[簡短原因]:  gpt4o性價比高，BuildMate B580顯卡、單通道高頻記憶體及1000W電源皆不合理。

Agent3: [對gpt4o評價]: 8
[對BuildMate評價]: 4
[簡短原因]: gpt4o較均衡，BuildMate H610搭DDR5及高時脈單通道記憶體、B580驅動及相容性堪慮、1000W電源過剩。

Coordinator: 
已達成共識，gpt4o較佳。BuildMate的H610主機板不適合搭配高時脈DDR5記憶體，B580顯卡驅動不穩定，1000W電源過剩。

{'Coordinator': {'messages': [HumanMessage(content='Agent1: [對gpt4o評價]: 7\n[對BuildMate評價]: 6\n[簡短原因]: gpt4o均衡，BuildMate記憶體單通道且顯卡驅動堪慮。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent2: [對gpt4o評價]: 8\n[對BuildMate評價]: 5\n[簡短原因]: gpt4o較均衡，BuildMate記憶體單通道、顯卡驅動問題、電源瓦數過高。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent3: [對gpt4o評價]: 8\n[對BuildMate評價]: 5\n[簡短原因]: gpt4o較均衡，BuildMate記憶體單通道、顯卡驅動問題、電源瓦數過高。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Coordinator: 已達成共識，gpt4o較佳。記憶體雙通道效能較佳、GTX1650驅動較穩定、650W電源較為合理。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent1: [對gpt4o評價]: 8\n[對BuildMate評價]: 5\n[簡短原因]: gpt4o均衡，BuildMate記憶體單通道、B580驅動及相容性堪慮、1000W電源過剩。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Agent2: [對gpt4o評價]: 8\n[對BuildMate評價]: 4\n[簡短原因]:  gpt4o性價比高，BuildMate B580顯卡、單通道高頻記憶體及1000W電源皆不合理。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content=